In [13]:
import pandas as pd
import numpy as np
import os
import networkx as nx
import random
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from pylab import rcParams
from sklearn.model_selection import train_test_split
rcParams['figure.figsize'] = 18, 15
%matplotlib inline

In [4]:
ideologies_dir = "./"
followers_dir = "../followers/"
geolocations_dir = "./"

In [5]:
ideals1 = pd.read_csv(ideologies_dir + "user-ideal-points-201807-000000000000.csv", usecols=["id_str", "theta"])
ideals2 = pd.read_csv(ideologies_dir + "user-ideal-points-201807-000000000001.csv",  usecols=["id_str", "theta"])
ideals3 =  pd.read_csv(ideologies_dir + "user-ideal-points-201807-000000000002.csv",  usecols=["id_str", "theta"])

ideals = ideals1.append(ideals2).append(ideals3)

In [6]:
user_ids = os.listdir(followers_dir)

In [7]:
geos1 = pd.read_csv(geolocations_dir + "already_located_random_users_machine00.csv", usecols=["user_id", "longitude", "latitude", "raw_location", "country"])
geos2 = pd.read_csv(geolocations_dir + "already_located_random_users_machine01.csv", usecols=["user_id", "longitude", "latitude", "raw_location", "country"])
geos3 = pd.read_csv(geolocations_dir + "already_located_random_users_machine02.csv", usecols=["user_id", "longitude", "latitude", "raw_location", "country"])

geos = geos1.append(geos2).append(geos3)

In [8]:
sample_size = 30

In [9]:
ideal_ids = ideals["id_str"]
geo_ids = geos["user_id"]
combined_ids = np.intersect1d(np.intersect1d(ideal_ids.values, geo_ids.values), user_ids)

In [10]:
random_users = random.sample(combined_ids.tolist(), sample_size)

In [11]:
result = []
failures = 0
for i, user_id in enumerate(random_users):
    try:
        followers = pd.read_csv(followers_dir + str(random_users[i]) + "/2019__10__" + str(random_users[i]) + ".csv")["user_id_followers"].values

        for follower in followers:
            result.append({
                "node_1": user_id,
                "node_2": follower
            })
    except Exception as e:
        failures += 1

print(str(failures) + " failures (total: " + str(len(random_users)) + ")")

1 failures (total: 30)


In [12]:
h = pd.DataFrame(result)
r = np.concatenate((h["node_1"].unique(),h["node_2"].unique()), axis=0)
rl = r.tolist()
h["node_1"] = h["node_1"].apply(lambda x: rl.index(x))
h["node_2"] = h["node_2"].apply(lambda x: rl.index(x))
h.to_csv("followerified_with_geo.csv", index=False)

In [ ]:
# g = nx.from_edgelist(h.values.tolist())
# nx.draw(g, with_labels=True)

In [17]:
embedding = pd.read_csv("follower-embedding20.csv")
embedding

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15
0,1.049803e-135,3.845377e-70,1.051272e-111,1.000000,2.002244e-115,2.748097e-33,3.099568e-42,2.601771e-08,5.113501e-46,7.292830e-22,3.421983e-04,1.235904e-05,3.762024e-04,4.714278e-16,2.029927e-15,8.632247e-24
1,6.291305e-192,2.000980e-107,1.198814e-158,1.000000,8.949510e-169,1.899186e-64,2.726435e-51,5.106335e-20,5.100849e-91,1.777599e-37,5.924143e-11,4.047241e-39,5.208689e-14,2.571439e-25,2.304615e-31,1.444237e-58
2,3.833054e-176,1.180748e-113,6.043453e-154,1.000000,5.056605e-159,3.114833e-68,1.010029e-51,7.189706e-20,3.585224e-88,1.940779e-37,1.364906e-10,3.926676e-41,1.839697e-13,5.282381e-24,6.913999e-32,2.812046e-60
3,1.595025e-192,1.179240e-107,3.131584e-159,1.000000,1.630649e-170,1.449859e-63,2.149302e-51,9.155831e-20,1.179333e-91,7.606157e-38,8.884254e-11,3.870469e-39,1.947175e-13,2.247506e-24,2.281899e-31,5.610604e-59
4,1.956287e-172,1.579778e-108,1.706288e-149,1.000000,2.938931e-155,3.471843e-64,3.084239e-51,2.007905e-19,8.306279e-84,3.185837e-37,6.187223e-11,2.371138e-37,2.072185e-13,2.508176e-24,4.239969e-31,3.494360e-55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25991,3.307305e-140,1.651135e-86,1.228504e-126,0.999997,1.794701e-119,1.203591e-05,3.336075e-125,2.327878e-104,8.515469e-67,2.412227e-03,2.462115e-95,2.224875e-23,5.559654e-85,4.394977e-118,3.150659e-34,4.539128e-18
25992,3.886732e-134,2.727706e-94,1.037211e-126,0.999997,5.314215e-118,7.774849e-06,8.226004e-127,7.198351e-109,8.565724e-68,2.407570e-03,4.544775e-96,9.857733e-27,5.048405e-84,1.741690e-113,1.248787e-36,4.078033e-21
25993,3.968319e-132,9.257175e-86,1.622799e-121,0.999997,1.030943e-113,1.276217e-05,3.139930e-121,7.614978e-102,3.084182e-62,2.411762e-03,2.478554e-91,9.597384e-20,3.650332e-81,2.218847e-111,3.565765e-32,4.547303e-17
25994,3.033227e-151,3.142960e-89,9.142298e-133,0.999997,1.372366e-130,1.075442e-05,2.605352e-129,4.417155e-109,2.358572e-72,2.411538e-03,1.513755e-102,1.118789e-26,3.992224e-91,3.856717e-126,6.877502e-38,3.838875e-20


In [18]:
grouped_embedding = embedding.groupby(lambda i: h.iloc[i]["node_1"] if i < h.shape[0] else None)

In [19]:
summed_embedding = grouped_embedding.apply(lambda x: np.log(x.sum()))
summed_embedding

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15
0,-89.187434,-25.212002,-29.051616,6.543059,-91.817486,-74.974400,-1.153979,0.293510,-6.414065,-0.529247,-0.383280,-11.301122,-0.525514,-0.210669,-3.639536,-23.145204
1,-7.211500,-84.220545,-0.515352,0.145121,-36.707121,-11.906915,5.775146,0.769956,-4.834091,-6.937058,-7.495495,-6.081750,-4.786189,-35.002159,-67.915064,-36.233831
2,-33.867844,0.621755,-2.176513,-2.418540,-34.618290,-132.937208,0.735210,-4.818624,-0.676820,-25.835067,6.655269,-7.636038,-8.059439,0.709078,-26.954167,-0.276975
3,-42.830795,0.343871,-0.465392,-1.882603,-38.530650,-12.262098,-2.294836,-0.222752,-0.871498,7.835633,0.710448,-6.583162,-0.345915,1.259039,-3.979804,-0.345070
4,-134.132214,-28.731557,-59.830079,-30.112902,-10.295203,-10.064268,5.690359,-43.713566,-52.149363,0.693147,-109.222669,-34.311263,-12.222036,-13.832090,-31.839463,-73.937068
5,8.342739,-28.019181,-0.089753,-2.910151,-2.652505,-13.237271,1.135508,-0.275752,-6.796503,-8.389342,-35.618018,-30.291859,1.844228,-8.126496,-0.427844,-0.409544
6,1.386298,0.722035,-0.416172,-26.010639,-8.877609,-53.792604,-4.803924,-13.946867,-35.933922,-6.928179,5.408117,-5.982167,-9.854884,0.287886,-4.506432,-26.086204
7,-56.536096,-3.934068,-82.205778,-8.845407,-27.014290,-100.401654,-78.745369,-6.229587,-9.580510,-41.734765,1.386281,-9.015562,-172.847260,-5.067847,-8.602714,4.852030
8,-33.999347,-52.253360,0.722988,-60.366825,-51.664370,-72.693734,-38.603581,-0.881218,6.480301,-0.991467,0.427898,0.913883,0.069341,0.448368,1.927110,1.609438
9,-137.619423,-8.592934,-4.816631,-12.660847,-8.244004,-8.742875,-225.565031,-6.755565,1.386266,-6.083330,-4.663319,-5.009595,-5.890327,-18.325320,4.263054,-7.676390


In [32]:
avg_embedding = grouped_embedding.apply(lambda x: x.mean())
avg_embedding

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15
0,2.653263e-42,1.614201e-14,3.470837e-16,9.977124e-01,1.912332e-43,3.948415e-36,4.531311e-04,1.926906e-03,2.353953e-06,8.463342e-04,9.793421e-04,1.775725e-08,8.494987e-04,1.163854e-03,3.773638e-05,1.275136e-13
1,2.277929e-06,8.183461e-40,1.843489e-03,3.568455e-03,3.529837e-19,2.081362e-08,9.944178e-01,6.665649e-03,2.454912e-05,2.997292e-06,1.714759e-06,7.049928e-06,2.575368e-05,1.941826e-18,9.870006e-33,5.666333e-19
2,2.504559e-18,2.384371e-03,1.452451e-04,1.140224e-04,1.182542e-18,2.362941e-61,2.670832e-03,1.034302e-05,6.507432e-04,7.714750e-15,9.947080e-01,6.181012e-07,4.047426e-07,2.601941e-03,2.519433e-15,9.706447e-04
3,9.885885e-23,5.565891e-04,2.477858e-04,6.006054e-05,7.286931e-21,1.865657e-09,3.977030e-05,3.158300e-04,1.650847e-04,9.980807e-01,8.030396e-04,5.459623e-07,2.792314e-04,1.389911e-03,7.375417e-06,2.794676e-04
4,1.874578e-61,1.116419e-15,3.482655e-29,2.804893e-16,1.134054e-07,1.428656e-07,9.932886e-01,3.477147e-22,7.544022e-26,6.711409e-03,1.233052e-50,4.213001e-18,1.651280e-08,3.300526e-09,4.989774e-17,2.602094e-35
5,9.984724e-01,1.612706e-16,2.173460e-04,1.294995e-05,1.675569e-05,4.238938e-10,7.400747e-04,1.804567e-04,2.657338e-07,5.403631e-08,8.080202e-20,1.661665e-17,1.503380e-03,7.028110e-08,1.549959e-04,1.578585e-04
6,1.746731e-02,8.989598e-03,2.880203e-03,2.207433e-14,6.090711e-07,1.898162e-26,3.579702e-05,3.829279e-09,1.082083e-18,4.278529e-06,9.747196e-01,1.101900e-05,2.292148e-07,5.823602e-03,4.819988e-05,2.046774e-14
7,2.118906e-27,1.482115e-04,1.506297e-38,1.091226e-06,1.403683e-14,1.886001e-46,4.794540e-37,1.492625e-05,5.231947e-07,5.678688e-21,3.030263e-02,9.204865e-07,6.498523e-78,4.769667e-05,1.390961e-06,9.696970e-01
8,2.594596e-18,3.065156e-26,3.117369e-03,9.179510e-30,5.523922e-26,4.067344e-35,2.597025e-20,6.267446e-04,9.866374e-01,5.613190e-04,2.320770e-03,3.773053e-03,1.621486e-03,2.368768e-03,1.039278e-02,7.564297e-03
9,2.278151e-62,2.472149e-06,1.079202e-04,4.230608e-08,3.504397e-06,2.127925e-06,1.456433e-100,1.552510e-05,5.333181e-02,3.040761e-05,1.258013e-04,8.898144e-05,3.688098e-05,1.466741e-10,9.470210e-01,6.181939e-06


In [45]:
def j(x):
    mapped_id = rl[int(x.name)]
    
    ideology = ideals[ideals["id_str"] == int(mapped_id)]
    geo = geos[geos["user_id"] == int(mapped_id)]
    
    x["mapped_id"] = mapped_id
    x['theta'] = ideology["theta"].iloc[0]
    x["longitude"] = geo["longitude"].iloc[0]
    x["latitude"] = geo["latitude"].iloc[0]
    x["raw_location"] = geo["raw_location"].iloc[0]
    x["country"] = geo["country"].iloc[0]
    return x

dataset = summed_embedding.apply(j, axis=1)

In [46]:
dataset

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_12,X_13,X_14,X_15,mapped_id,theta,longitude,latitude,raw_location,country
0,-89.187434,-25.212002,-29.051616,6.543059,-91.817486,-74.974400,-1.153979,0.293510,-6.414065,-0.529247,...,-0.525514,-0.210669,-3.639536,-23.145204,230015947,-0.390792,79.00000,22.00000,indy,India
1,-7.211500,-84.220545,-0.515352,0.145121,-36.707121,-11.906915,5.775146,0.769956,-4.834091,-6.937058,...,-4.786189,-35.002159,-67.915064,-36.233831,3408070107,2.183832,-96.61527,33.19762,"mckinney, tx",United States
2,-33.867844,0.621755,-2.176513,-2.418540,-34.618290,-132.937208,0.735210,-4.818624,-0.676820,-25.835067,...,-8.059439,0.709078,-26.954167,-0.276975,57924597,0.256460,36.81667,-1.28333,"nairobi, kenya",Kenya
3,-42.830795,0.343871,-0.465392,-1.882603,-38.530650,-12.262098,-2.294836,-0.222752,-0.871498,7.835633,...,-0.345915,1.259039,-3.979804,-0.345070,558705332,-0.421356,-43.18223,-22.90642,"rio de janeiro, brasil",Brazil
4,-134.132214,-28.731557,-59.830079,-30.112902,-10.295203,-10.064268,5.690359,-43.713566,-52.149363,0.693147,...,-12.222036,-13.832090,-31.839463,-73.937068,459718220,0.022730,-111.65853,40.23384,"utah, usa",United States
5,8.342739,-28.019181,-0.089753,-2.910151,-2.652505,-13.237271,1.135508,-0.275752,-6.796503,-8.389342,...,1.844228,-8.126496,-0.427844,-0.409544,279664901,0.634024,-74.00597,40.71427,"new york, ny",United States
6,1.386298,0.722035,-0.416172,-26.010639,-8.877609,-53.792604,-4.803924,-13.946867,-35.933922,-6.928179,...,-9.854884,0.287886,-4.506432,-26.086204,42680994,1.405332,4.47917,51.92250,rotterdam,Netherlands
7,-56.536096,-3.934068,-82.205778,-8.845407,-27.014290,-100.401654,-78.745369,-6.229587,-9.580510,-41.734765,...,-172.847260,-5.067847,-8.602714,4.852030,327552212,0.002953,-118.44925,34.15112,sherman oaks,United States
8,-33.999347,-52.253360,0.722988,-60.366825,-51.664370,-72.693734,-38.603581,-0.881218,6.480301,-0.991467,...,0.069341,0.448368,1.927110,1.609438,24648846,-0.955339,-121.49440,38.58157,sacramento,United States
9,-137.619423,-8.592934,-4.816631,-12.660847,-8.244004,-8.742875,-225.565031,-6.755565,1.386266,-6.083330,...,-5.890327,-18.325320,4.263054,-7.676390,290376098,-0.473496,-121.21605,37.79743,"manteca, ca",United States


In [47]:
dataset.to_csv("avg_embedding.csv", index=False)

In [48]:
X = dataset[dataset.columns.difference(['mapped_id', 'theta', "country","raw_location", "latitude", "longitude"])]
X

,X_0,X_1,X_10,X_11,X_12,X_13,X_14,X_15,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9
0,-89.187434,-25.212002,-0.383280,-11.301122,-0.525514,-0.210669,-3.639536,-23.145204,-29.051616,6.543059,-91.817486,-74.974400,-1.153979,0.293510,-6.414065,-0.529247
1,-7.211500,-84.220545,-7.495495,-6.081750,-4.786189,-35.002159,-67.915064,-36.233831,-0.515352,0.145121,-36.707121,-11.906915,5.775146,0.769956,-4.834091,-6.937058
2,-33.867844,0.621755,6.655269,-7.636038,-8.059439,0.709078,-26.954167,-0.276975,-2.176513,-2.418540,-34.618290,-132.937208,0.735210,-4.818624,-0.676820,-25.835067
3,-42.830795,0.343871,0.710448,-6.583162,-0.345915,1.259039,-3.979804,-0.345070,-0.465392,-1.882603,-38.530650,-12.262098,-2.294836,-0.222752,-0.871498,7.835633
4,-134.132214,-28.731557,-109.222669,-34.311263,-12.222036,-13.832090,-31.839463,-73.937068,-59.830079,-30.112902,-10.295203,-10.064268,5.690359,-43.713566,-52.149363,0.693147
5,8.342739,-28.019181,-35.618018,-30.291859,1.844228,-8.126496,-0.427844,-0.409544,-0.089753,-2.910151,-2.652505,-13.237271,1.135508,-0.275752,-6.796503,-8.389342
6,1.386298,0.722035,5.408117,-5.982167,-9.854884,0.287886,-4.506432,-26.086204,-0.416172,-26.010639,-8.877609,-53.792604,-4.803924,-13.946867,-35.933922,-6.928179
7,-56.536096,-3.934068,1.386281,-9.015562,-172.847260,-5.067847,-8.602714,4.852030,-82.205778,-8.845407,-27.014290,-100.401654,-78.745369,-6.229587,-9.580510,-41.734765
8,-33.999347,-52.253360,0.427898,0.913883,0.069341,0.448368,1.927110,1.609438,0.722988,-60.366825,-51.664370,-72.693734,-38.603581,-0.881218,6.480301,-0.991467
9,-137.619423,-8.592934,-4.663319,-5.009595,-5.890327,-18.325320,4.263054,-7.676390,-4.816631,-12.660847,-8.244004,-8.742875,-225.565031,-6.755565,1.386266,-6.083330


In [49]:
y = dataset[["theta"]]
y

,theta
0,-0.390792
1,2.183832
2,0.256460
3,-0.421356
4,0.022730
5,0.634024
6,1.405332
7,0.002953
8,-0.955339
9,-0.473496


In [ ]:
y_multi = dataset[["theta", "latitude", "longitude"]]

In [50]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y_multi, test_size=0.33, random_state=42)

In [51]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [44]:
model.score(X_test, y_test)

-0.8590394584423395

In [58]:
model.score(X_test, y_test)

-0.05136216486570677

In [57]:
model.predict(X_test).tolist()

[[-1.236923819250836],
 [0.5889489938243663],
 [0.3138783707380951],
 [0.27335657557303306],
 [-1.4266281475430262],
 [0.08587662074008717],
 [-0.05274854116994116],
 [-1.115166970276979],
 [-3.41309629878513],
 [-0.14660547400273294],
 [-1.3035351525927679],
 [-0.69262344205512],
 [0.9388193469123041]]

In [56]:
tree_model = RandomForestRegressor()
tree_model.fit(X_train, y_train)

,theta
33,-0.007834
36,1.495228
4,0.022730
13,-0.843868
30,-1.025458
26,0.707738
6,1.405332
27,-0.092337
24,-1.748223
15,1.953698


In [ ]:
tree_model.score(X_test, y_test)

In [ ]:
# http://cig.fi.upm.es/articles/2015/Borchani-2015-WDMKD.pdf